In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from config import DATABASE_URI
from models import Security, Earnings_release
from sqlalchemy import Column, Table
from scrapping_sources.Macrotrend import Macrotrend
from scrapping_sources.Nasdaq import Nasdaq
from models import Base
import datetime
import pandas as pd

In [20]:
USERNAME        = "postgres"
PASSWORD        = "postgres5432"
HOST            = "localhost"
PORT            = "4567" # os.getenv("POSTGRES_PORT")
DATABASE_NAME   = "postgres"
DATABASE_URI = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE_NAME}"\

session = sessionmaker(bind=create_engine(DATABASE_URI))

In [34]:
# from config import DATABASE_URI
from models import Earnings_release, Security, Base
from scrapping_sources.Macrotrend import Macrotrend
# from earnings_release.earnings_release import last_period_db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import pandas as pd
import datetime

DAY_VARIABLE = 30
look_back_date = str(datetime.date.today() - datetime.timedelta(days=DAY_VARIABLE))

engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)
s = Session()

querry = s.query(Security.ticker, Security.id).all()
security_map = {querry[i][0]: querry[i][1] for i, v in zip(range(len(querry)), range(len(querry)))}

statements = [
    'income-statement',
    'balance-sheet',
    'cash-flow-statement',
    'financial-ratios'
]
time_format = [
    'annual',
    'quarterly'
]

M = Macrotrend()

def convert_to_ending_period_format(date):
    """Returns """
    result = datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime("%Y-%m")
    return result

def last_period_db(ticker):
    try:
        r = s.query(Base.metadata.tables['income_statement_quarterly'].columns['date']).where(
            Base.metadata.tables['income_statement_quarterly'].columns['ticker'] == str(ticker)
        ).all()
        r.sort(reverse=True)
        r =  str(r[0][0])
        return convert_to_ending_period_format(r)
    except:
        return None

def fetch_all_statement(ticker, stmnt, time_format):

    r = s.query(Base.metadata.tables[f'{stmnt.replace("-", "_")}_{time_format}']).where(\
            Base.metadata.tables[f'{stmnt.replace("-", "_")}_{time_format}'].columns['ticker'] ==str(ticker)).all()
    return r

def statement_table_log(ticker, statement, time_format, status):

    try:
        security_id= security_map[f"{ticker}"]
        load = pd.DataFrame(
        [
            {
                "date"          : str(datetime.date.today()),
                "ticker"        : f"{ticker}",
                "security_id"   : security_id,
                "statement"     : f"{statement}",
                "time_format"   : time_format,
                "status"        : status
              }
        ]
    )

        load.to_sql(con=engine, name="statements_table_log", if_exists="append", index=False)
    except:
        security_id = None

    load = pd.DataFrame(
        [
            {
                "date"          : str(datetime.date.today()),
                "ticker"        : f"{ticker}",
                "security_id"   : security_id,
                "statement"     : f"{statement}",
                "time_format"   : time_format,
                "status"        : status
              }
        ]
    )

    load.to_sql(con=engine, name="statements_table_log", if_exists="append", index=False)

def update_db(ticker, stmnt, t_format):

    latest = M.arrange_data(ticker, stmnt, t_format)
    print("LATEST", latest)
    in_database = pd.DataFrame(fetch_all_statement(ticker, stmnt, t_format))
    print(f"IN DATABASE {ticker} {stmnt} {t_format}", in_database)

    if not in_database.empty:

        try:
            in_database.columns = ['id','date','statement','ticker','security_id','line_item', 'amount']
            in_database['security_id'] = in_database.ticker.map(security_map)
            in_database = M.move_column(in_database, 'security_id', 3)
            print(in_database)
            latest['security_id'] = latest.ticker.map(security_map)
            latest['amount'] = pd.to_numeric(latest['amount'])
            convert_dict = {
                'date': str,
                'statement': str,
                'ticker': str,
                'security_id': int,
                'line_item': str,
                # 'amount': float
            }

            in_database = in_database.astype(convert_dict)
            latest = latest.astype(convert_dict)

        except:

            print("Something went wrong in the dataframe creation")

        if isinstance(in_database, pd.DataFrame) and isinstance(latest, pd.DataFrame):

            if latest.shape[0] == in_database.shape[0]:

                print(f"Security {ticker} is already up to date")
                statement_table_log(ticker, stmnt, t_format, status="up to date")

            else:

                in_database['date'] = pd.to_datetime(in_database['date'])
                indb_date_set = set(in_database.date.values)
                latest['date'] = pd.to_datetime(latest.date)
                update_date_set = set(latest.date.values)
                update = latest[latest.date.isin(list(update_date_set - indb_date_set))]
                # UPDATE THE CORRESPONDING TABLE
                update.to_sql(con = engine, name=f"{stmnt.replace('-','_')}_{time_format}", if_exists='append', index=False)
                # UPDATE THE STATEMENTS TABLE LOG
                statement_table_log(ticker, stmnt, t_format, status="updated")
                
        elif not isinstance(in_database, pd.DataFrame):

            statement_table_log(
                ticker,
                stmnt, 
                t_format, 
                status=f"{ticker} Not in Database"
            )

        else:

            statement_table_log(
                ticker,
                stmnt, 
                t_format, 
                status=f"{ticker} no data availble on M"
            )

    else:
        latest['security_id'] = latest.ticker.map(security_map)
        latest['amount'] = pd.to_numeric(latest['amount'])
        convert_dict = {
            'date': str,
            'statement': str,
            'ticker': str,
            'security_id': int,
            'line_item': str,
            # 'amount': float
            }
        latest = latest.astype(convert_dict)
        latest['date'] = pd.to_datetime(latest.date)
        latest.to_sql(con = engine, name=f"{stmnt.replace('-','_')}_{time_format}", if_exists='append', index=False)
                # UPDATE THE STATEMENTS TABLE LOG
        statement_table_log(ticker, stmnt, t_format, status="updated")


r = s.query(Earnings_release.__table__).filter(Earnings_release.release_date >= look_back_date).all()
earnings_df = pd.DataFrame(r)
print(earnings_df)
earnings_df.columns = Earnings_release.__table__.columns.keys()
df = earnings_df[earnings_df['last_period_DB'].notna()]

# 1 - Remove all the rows that dont have data on Trend
df = df[df['last_period_M'].notna()]

# 2 - Remove all the rows where DB = N
df = df[df['last_period_DB'] != df.last_period_N]

for row in df.iterrows():

    id_ = row[1][0]
    print(id_)
    ticker = row[1][3]
    last_period_M_on_record = row[1][7]
    on_DB = row[1][6]
    print(on_DB)
    M_latest = M.latest_ending_period_available(ticker)
    print(last_period_M_on_record == M_latest)

    if last_period_M_on_record != M_latest:
        print("MARKER 1")

        # UPDATE EARNINGS TABLE M
        s.query(Earnings_release.__table__).\
            filter(Earnings_release.id == id_).\
            update({"last_period_M": f"{M_latest}"})
        s.commit()

        # GET THE STATEMENTS FROM MACROTREND AND POPULATE DB
        for stmnt in statements:
            for t_format in time_format:
                update_db(ticker, stmnt, t_format)
                # statement_table_log(ticker, stmnt, t_format, status="updated")

        #  VERIFY, IF SUCCESFUL, UPDATE EARNINGS TABLE AND STATEMENTS TABLE LOG
        if last_period_db(ticker) == on_DB:
            print("MARKER 2")
            s.query(Earnings_release.__table__).\
                filter(Earnings_release.id == id_).\
                update({"last_period_DB": f"{last_period_db(ticker)}"})
            s.commit()


        pass

    # IF THE DATE ON THE DATABASE IS NOT THE SAME AS THE DATE ON M
    # POPULATE THE DATABASE WITH THE MISSING DATA
    print(on_DB)
    if on_DB != last_period_M_on_record:
        print("MARKER 3")
        for stmnt in statements:
            for t_format in time_format:
                update_db(ticker, stmnt, t_format)
                # statement_table_log(ticker, stmnt, t_format, status="updated")
    
    else:
        pass

s.close_all()

       0           1           2     3       4        5        6        7
0    359  2021-09-26  2021-09-24  JOBS  4133.0  2021-06  2021-03  2021-03
1    360  2021-09-26  2021-09-24  MVST  4999.0  2021-06  2021-06  2021-06
2    361  2021-09-26  2021-09-24  UXIN  7667.0  2021-06  2021-03  2021-03
3    362  2021-09-26  2021-09-24   SVA     NaN  2021-06     None  2020-12
4    363  2021-09-26  2021-09-24   VWE  7927.0  2021-06  2021-03  2021-03
..   ...         ...         ...   ...     ...      ...      ...      ...
353  712  2021-09-26  2021-08-30   SOL  6834.0  2021-06  2021-06  2021-06
354  713  2021-09-26  2021-08-30   NAT  5044.0  2021-06  2021-06  2021-06
355  714  2021-09-26  2021-08-30  PRTG  5926.0  2021-06  2021-06  2021-06
356  715  2021-09-26  2021-08-30  SERA     NaN  2021-06     None  2021-06
357  716  2021-09-26  2021-08-30  LIZI  4483.0  2021-06  2021-06  2021-06

[358 rows x 8 columns]
359
2021-03
JOBS <Response [200]>
True
2021-03
361
2021-03
UXIN <Response [200]>
True
20

/tmp/ipykernel_35732/1150489090.py:226: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [31]:
df = pd.DataFrame(fetch_all_statement('BGI', "balance-sheet", "quarterly"))
df.empty

True

In [23]:
str(datetime.datetime.today())

'2021-09-22 17:48:27.818873'

In [33]:
s.query(Earnings_release.__table__).\
       filter(Earnings_release.id == 38).\
       update({"last_period_M": '2021-03'})
s.commit()

In [10]:
last_period_db('AAPL')

'2021-06'

In [24]:
[datetime.date.today() - datetime.timedelta(days= i) for i in range(0,30)]

[datetime.date(2021, 9, 22),
 datetime.date(2021, 9, 21),
 datetime.date(2021, 9, 20),
 datetime.date(2021, 9, 19),
 datetime.date(2021, 9, 18),
 datetime.date(2021, 9, 17),
 datetime.date(2021, 9, 16),
 datetime.date(2021, 9, 15),
 datetime.date(2021, 9, 14),
 datetime.date(2021, 9, 13),
 datetime.date(2021, 9, 12),
 datetime.date(2021, 9, 11),
 datetime.date(2021, 9, 10),
 datetime.date(2021, 9, 9),
 datetime.date(2021, 9, 8),
 datetime.date(2021, 9, 7),
 datetime.date(2021, 9, 6),
 datetime.date(2021, 9, 5),
 datetime.date(2021, 9, 4),
 datetime.date(2021, 9, 3),
 datetime.date(2021, 9, 2),
 datetime.date(2021, 9, 1),
 datetime.date(2021, 8, 31),
 datetime.date(2021, 8, 30),
 datetime.date(2021, 8, 29),
 datetime.date(2021, 8, 28),
 datetime.date(2021, 8, 27),
 datetime.date(2021, 8, 26),
 datetime.date(2021, 8, 25),
 datetime.date(2021, 8, 24)]

In [8]:
statements = [
'income_statement',
'balance_sheet',
'cash_flow_statement',
'financial_ratios'
]
time_format = [
    'annual',
    'quarterly'
]
def has_f_data_in_db(ticker):

    data_boolean_list = []
    for stmnt in statements:
        for t_format in time_format:
            q = s.query(Base.metadata.tables[f'{stmnt}_{t_format}']).where(
                Base.metadata.tables[f'{stmnt}_{t_format}'].columns['ticker'] ==str(ticker)).all()
            print(q)
            if len(q) == 0:
                data_boolean_list.append(False)
            else:
                data_boolean_list.append(True)
    return any(data_boolean_list)

def fetch_all_statement(ticker, stmnt, time_format):

    r = s.query(Base.metadata.tables[f'{stmnt}_{time_format}']).where(\
            Base.metadata.tables[f'{stmnt}_{time_format}'].columns['ticker'] ==str(ticker)).all()
    return r

pd.DataFrame(fetch_all_statement('AAPL', 'balance_sheet', 'annual'))

            

,0,1,2,3,4,5,6
0,3917,2020-09-30,BS-A,AAPL,17,Receivables,37445.0
1,3918,2020-09-30,BS-A,AAPL,17,Inventory,4061.0
2,3919,2020-09-30,BS-A,AAPL,17,Pre-Paid Expenses,None
3,3920,2020-09-30,BS-A,AAPL,17,Other Current Assets,11264.0
4,3921,2020-09-30,BS-A,AAPL,17,Total Current Assets,143713.0
...,...,...,...,...,...,...,...
347,4264,2005-09-30,BS-A,AAPL,17,Retained Earnings (Accumulated Deficit),3925.0
348,4265,2005-09-30,BS-A,AAPL,17,Comprehensive Income,None
349,4266,2005-09-30,BS-A,AAPL,17,Other Share Holders Equity,None
350,4267,2005-09-30,BS-A,AAPL,17,Share Holder Equity,7428.0


In [88]:

M = Macrotrend()

def has_data_trend(ticker):
    _ = []

    for stmnt in statements:
        for t_format in time_format:
            r = M.arrange_data(ticker, stmnt.replace('_', '-'), t_format)
            if r is not None:
                _.append(True)
    print(_)
    return any(_)


In [104]:
N = Nasdaq()

def convert_date_N(date):
    return datetime.datetime.strptime(str(date), '%b/%Y').strftime("%Y-%m")
# def last_period_nasdaq(ticker):
data = N.earnings_release(datetime.date.today())
data['fiscalQuarterEnding'].map(convert_date_N)
# lambda x: datetime.datetime.strptime(str(x), '%b/%Y').strftime("%Y-%m")

# datetime.datetime.strptime('Jun/2021', '%b/%Y').strftime("%Y-%m")

0     2021-06
1     2021-06
2     2021-06
3     2021-06
4     2021-06
5     2021-06
6     2021-06
7     2021-06
8     2021-06
9     2021-06
10    2021-06
11    2021-06
12    2021-06
13    2021-06
14    2021-06
15    2021-06
16    2021-06
17    2021-06
18    2021-06
19    2021-06
20    2021-06
Name: fiscalQuarterEnding, dtype: object

In [23]:
querry = s.query(Security.ticker, Security.id).all()
security_id_mapper = {querry[i][0]: querry[i][1] for i, v in zip(range(len(querry)), range(len(querry)))}

In [122]:
r = s.query(Base.metadata.tables['income_statement_quarterly'].columns['date']).where(
    Base.metadata.tables['income_statement_quarterly'].columns['ticker'] == str('AAPL')
).all()
r.sort(reverse=True)
r = str(r[0][0])

def convert_to_ending_period_format( date):
    """Returns """
    result = datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime("%Y-%m")
    return result

convert_to_ending_period_format(r)

'2021-06'

In [123]:
M.latest_ending_period_available('AAPL', 'balance-sheet', 'annual')

AAPL <Response [200]>


'2020-09'

In [25]:
start =time.perf_counter()
finviz_table = None

while finviz_table is None and (time.perf_counter() - start) < 20:
    print((time.perf_counter() - start))
    if (time.perf_counter() - start) >5:
        finviz_table = 1
print(finviz_table)

NameError: name 'time' is not defined

In [ ]:
engine = create_engine(DATABASE_URI)
Session = sessionmaker(bind=engine)
s = Session()

In [ ]:
q = s.query(Security.ticker).all()

In [ ]:
[i for i in range(0,75,10)]
# COMPARE WITH MYSQL OUTPUT. MAKE SURE THEY ARE THE SAME

[0, 10, 20, 30, 40, 50, 60, 70]

In [ ]:
my_list = [i for i in range(0,75)]

In [ ]:
[my_list[i:i + 10] for i in range(0, len(my_list), 10)]

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74]]

In [ ]:
def chunks(lst, n):
    chunk_list = []
    for i in range(0, len(lst), n):
        chunk_list.append(lst[i:i + n])
    return chunk_list

In [ ]:
chunks(my_list, 10)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 [70, 71, 72, 73, 74]]

In [ ]:
s.close_all()

/tmp/ipykernel_534434/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [ ]:
r = s.query(Security.ticker).all()

In [ ]:
tickers =[i[0] for i in r][:100]

In [ ]:
scrapper = Macrotrend()
scrapper.generate_statement_table_multi_threading(tickers, "balance-sheet", "annual")

AAAU <Response [404]>
No data available for AAAU, balance-sheet, annual
AADR <Response [404]>
No data available for AADR, balance-sheet, annual
AADI <Response [200]>
AAIC <Response [200]>
AAL <Response [200]>
AA <Response [200]>
AACG <Response [200]>
AACIU <Response [404]>
No data available for AACIU, balance-sheet, annual
AAC <Response [200]>
A <Response [200]>
AAMC <Response [200]>
AAME <Response [200]>
AAN <Response [200]>
AAPL <Response [200]>
AAON <Response [200]>
AAP <Response [200]>
AAOI <Response [200]>
AAT <Response [200]>
AAQC <Response [200]>
No data available for AAQC, balance-sheet, annual
AATC <Response [200]>
AAWW <Response [200]>
AB <Response [200]>
ABB <Response [200]>
AAU <Response [200]>
AAXJ <Response [404]>
No data available for AAXJ, balance-sheet, annual
ABBV <Response [200]>
ABCB <Response [200]>
ABCL <Response [200]>
ABC <Response [200]>
ABCM <Response [200]>
ABEO <Response [200]>
ABEQ <Response [404]>
No data available for ABEQ, balance-sheet, annual
ABEV <Res

,date,statement,ticker,security_id,line_item,amount
0,2020-12-31,BS-A,AADI,None,Receivables,NaN
1,2020-12-31,BS-A,AADI,None,Inventory,NaN
2,2020-12-31,BS-A,AADI,None,Pre-Paid Expenses,0.5102
3,2020-12-31,BS-A,AADI,None,Other Current Assets,1.6039
4,2020-12-31,BS-A,AADI,None,Total Current Assets,44.7190
...,...,...,...,...,...,...
39,2019-12-31,BS-A,ACVA,None,Retained Earnings (Accumulated Deficit),-125.9580
40,2019-12-31,BS-A,ACVA,None,Comprehensive Income,-0.0010
41,2019-12-31,BS-A,ACVA,None,Other Share Holders Equity,NaN
42,2019-12-31,BS-A,ACVA,None,Share Holder Equity,205.3260


In [ ]:
import time
import pandas as pd
import requests, json, re
from bs4 import BeautifulSoup as bs
import datetime
import concurrent.futures
from itertools import repeat



class Macrotrend():

    # def __init__(self): #self, ticker=None, statement=None, time_format=None, **kwargs):
    #     # super().__init__(**kwargs)
    #     # self.ticker         = ticker
    #     # self.statement      = statement
    #     # self.time_format    = time_format
    #     pass
        
    def get_statement(self, ticker, statement, time_format):

        statement_data = None

        if time_format == 'annual':

            try:

                r = requests.get('https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)

                # return pd.DataFrame(results, columns = fields)
                statement_data = pd.DataFrame(results, columns = fields)

                return statement_data
        
            except:
                print(f"No data available for {ticker}, {statement}, annual")
                pass

            pass

        else:

            try:

                r = requests.get(f'https://www.macrotrends.net/stocks/charts/' + ticker + '/' + ticker + '/' + statement)
                company_name = r.url.split('/')[6]
                r = requests.get(f'https://www.macrotrends.net/stocks/charts/'+ ticker + '/' + company_name + '/' + statement + '?freq=Q')
                print(ticker, r)
                p = re.compile(r'var originalData = (.*);')
                p2 = re.compile(r'datafields:[\s\S]+(\[[\s\S]+?\]),')
                p3 = re.compile(r'\d{4}-\d{2}-\d{2}')
                data = json.loads(p.findall(r.text)[0])
                s = re.sub('\r|\n|\t|\s','',p2.findall(r.text)[0])
                fields = p3.findall(s)
                fields.insert(0, 'field_name') # only headers of interest.
                results = []

                for item in data: #loop initial list of dictionaries
                    row = {}
                    for f in fields: #loop keys of interest to extract from current dictionary
                        if f == 'field_name':  #this is an html value field so needs re-parsing
                            soup2 = bs(item[f],'lxml')
                            row[f] = soup2.select_one('a,span').text
                        else:
                            row[f] = item[f]
                    results.append(row)

                # return pd.DataFrame(results, columns = fields)
                statement_data = pd.DataFrame(results, columns = fields)
                return statement_data
    
            except:
                print(f"No data available for {ticker}, {statement}, quarterly")
                pass

    def move_column(self, df, column, pos):
    
        col = df[column]
        df.drop(columns=[column],inplace = True)
        df.insert(pos, column, col)
        
        return df
    
    def generate_statement_key(self, statement, time_format):

        report_formats  = {'quarterly'  : 'Q',
                            'annual'    : 'A'}
        statements      = {'income-statement'       :'IS',
                            'balance-sheet'         :'BS',
                            'cash-flow-statement'   :'CF',
                            'financial-ratios'      :'R'}

        statement_key = statements[statement] + '-' + report_formats[time_format]

        return statement_key

    def arrange_data(self, ticker, statement, time_format):

        data_dict = []
        # no_data_tick = []
        report_formats = {'quarterly' : 'Q',
                        'annual': 'A'}
        statements = {'income-statement':'IS',
                    'balance-sheet':'BS',
                    'cash-flow-statement':'CF',
                    'financial-ratios':'R'}


        df = self.get_statement(ticker, statement, time_format)

        if isinstance(df, pd.DataFrame):

            for i in df.columns[1:]:

                keys = df[df.columns[0]]
                data = df[i].values

                data = {k:v for k,v in zip(keys, data)}
                date = i

                data['date'] = date
                data['ticker'] = ticker
                data['statement_format'] = statements[statement] + '-' + report_formats[time_format]

                data_dict.append(data)

            df = pd.DataFrame(data_dict)
            df = self.move_column(df, 'date', 0)
            df = self.move_column(df, 'ticker', 1)
            df = self.move_column(df, 'statement_format', 2)

            my_series = []

            line_item = []
            amount = []
            date = []
            ticker = []
            security_id = []
            statement_format = []

            for i in df.iterrows():

                serie = pd.Series(i)[1]

                my_series.append(serie)
                [statement_format.append(serie[2]) for i in range(len(serie.index.values[4:]))]
                # [security_id.append(serie[3]) for i in range(len(serie.index.values[4:]))]
                [line_item.append(i) for i in  serie.index.values[4:]]
                [amount.append(i) for i in serie.values[4:]]
                [date.append(serie[0]) for i in  range(len(serie.index.values[4:]))]
                [ticker.append(serie[1]) for i in range(len(serie.index.values[4:]))]

            data = pd.DataFrame([date, statement_format, ticker, security_id,line_item, amount]).T
            data.columns = ['date','statement','ticker','security_id','line_item','amount']
            return data

        else:
            
            pass

    def generate_statement_table_multi(self, ticker_list, statement, time_format):
        start = time.perf_counter()
        table   = []
        # no_income_statement_list = []

        for i in ticker_list:
            df = self.arrange_data(i, statement, time_format)

            if isinstance(df, pd.DataFrame):
                table.append(df)
            else:
                pass
        
        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])
        finish = time.perf_counter()
        print(finish-start)
        return table_concat

    def generate_statement_table_multi_threading(self, ticker_list, statement, time_format):
        start = time.perf_counter()
        table = []

        def create_table(ticker, statement, time_format):

            df = self.arrange_data(ticker, statement, time_format)
            if isinstance(df, pd.DataFrame):
                table.append(df)


        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

            executor.map(create_table, ticker_list, repeat(statement), repeat(time_format))

        table_concat = pd.concat(table)
        table_concat['amount'] =  pd.to_numeric(table_concat['amount'])
        finish = time.perf_counter()
        print(finish-start)
        return table_concat

    def generate_statement_table(self, ticker, statement, time_format):

        pass
        # TRY SIMPLY GOING WITH THE OLD SCRIPT BY UPLOADING IN CHUNKS
    
    def latest_ending_period_available(self, ticker, statement, time_format):

        try:
            df = self.get_statement(ticker, statement, time_format)
            raw_date = df.columns[1]
            converted_date = datetime.datetime.strptime(str(raw_date), '%Y-%m-%d').strftime("%Y-%m")
        except:
            converted_date = None

        return converted_date

scrapper = Macrotrend()

print(scrapper.latest_ending_period_available('AAPL', 'balance-sheet', 'quarterly'))
    

AAPL <Response [200]>
2021-06


In [ ]:
scrapper = Macrotrend()

In [ ]:
scrapper.generate_statement_table_multi_threading(tickers, "balance-sheet", "quarterly")

A <Response [200]>
AACIU <Response [404]>
No data available for AACIU, balance-sheet, quarterly
AAAU <Response [404]>
No data available for AAAU, balance-sheet, quarterly
AACG <Response [200]>
AAC <Response [200]>
AADI <Response [200]>
AAL <Response [200]>
AAIC <Response [200]>AA 
AADR <Response [404]>
No data available for AADR, balance-sheet, quarterly
<Response [200]>
AAME <Response [200]>
AAMC <Response [200]>
AAOI <Response [200]>
AAN <Response [200]>
AAPL <Response [200]>
AAON <Response [200]>
AAP <Response [200]>
AATC <Response [200]>
AAU <Response [200]>
AAQC <Response [200]>
AAWW <Response [200]>
AAT <Response [200]>
AAXJ <Response [404]>
No data available for AAXJ, balance-sheet, quarterly
AB <Response [200]>
ABBV <Response [200]>
ABB <Response [200]>
ABC <Response [200]>
ABCB <Response [200]>
ABCL <Response [200]>
ABEQ <Response [404]>
No data available for ABEQ, balance-sheet, quarterly
ABCM <Response [200]>
ABEO <Response [200]>
ABEV <Response [200]>
ABGI <Response [200]>


,date,statement,ticker,security_id,line_item,amount
0,2021-07-31,BS-Q,A,None,Receivables,1122.000
1,2021-07-31,BS-Q,A,None,Inventory,818.000
2,2021-07-31,BS-Q,A,None,Pre-Paid Expenses,NaN
3,2021-07-31,BS-Q,A,None,Other Current Assets,264.000
4,2021-07-31,BS-Q,A,None,Total Current Assets,3632.000
...,...,...,...,...,...,...
127,2019-12-31,BS-Q,ACVA,None,Retained Earnings (Accumulated Deficit),-125.958
128,2019-12-31,BS-Q,ACVA,None,Comprehensive Income,NaN
129,2019-12-31,BS-Q,ACVA,None,Other Share Holders Equity,-0.001
130,2019-12-31,BS-Q,ACVA,None,Share Holder Equity,205.326


In [ ]:
table = []

def create_table(ticker, statement, time_format):

    df = scrapper.arrange_data(ticker, statement, time_format)
    if isinstance(df, pd.DataFrame):
        table.append(df)


with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:

    executor.map(create_table, tickers, repeat('balance-sheet'), repeat("annual"))

# table
table_concat = pd.concat(table)
table_concat['amount'] =  pd.to_numeric(table_concat['amount'])


AAIC <Response [200]>
AAC <Response [200]>
AAAU <Response [404]>
No data available for AAAU, balance-sheet, annual
AADI <Response [200]>
AAL <Response [200]>
AADR <Response [404]>
No data available for AADR, balance-sheet, annual
A <Response [200]>
AA <Response [200]>
AACIU <Response [404]>
No data available for AACIU, balance-sheet, annual
AACG <Response [200]>
AAMC <Response [200]>
AAME <Response [200]>
AAN <Response [200]>
AAP <Response [200]>
AAOI <Response [200]>
AAON <Response [200]>
AAQCAAXJ <Response [404]>
No data available for AAXJ, balance-sheet, annual
 AAT <Response [200]>
AAU <Response [200]>
AAWW <Response [200]>
AATC <Response [200]>
AAPL <Response [200]><Response [200]>
No data available for AAQC, balance-sheet, annual

ABC <Response [200]>
ABB <Response [200]>
AB <Response [200]>
ABBV <Response [200]>
ABCBABEQ <Response [200]>
 <Response [404]>
No data available for ABEQ, balance-sheet, annual
ABCL <Response [200]>
ABEO <Response [200]>
ABCM <Response [200]>
ABEV <Res

In [ ]:
table_concat

,date,statement,ticker,security_id,line_item,amount
0,2020-12-31,BS-A,AAMC,None,Receivables,3.414
1,2020-12-31,BS-A,AAMC,None,Inventory,NaN
2,2020-12-31,BS-A,AAMC,None,Pre-Paid Expenses,3.328
3,2020-12-31,BS-A,AAMC,None,Other Current Assets,48.249
4,2020-12-31,BS-A,AAMC,None,Total Current Assets,96.614
...,...,...,...,...,...,...
39,2019-12-31,BS-A,ACVA,None,Retained Earnings (Accumulated Deficit),-125.958
40,2019-12-31,BS-A,ACVA,None,Comprehensive Income,-0.001
41,2019-12-31,BS-A,ACVA,None,Other Share Holders Equity,NaN
42,2019-12-31,BS-A,ACVA,None,Share Holder Equity,205.326


In [ ]:
scrapper.get_statement('AAPL', 'balance-sheet', 'annual')

AAPL <Response [200]>


,field_name,2020-09-30,2019-09-30,2018-09-30,2017-09-30,2016-09-30,2015-09-30,2014-09-30,2013-09-30,2012-09-30,2011-09-30,2010-09-30,2009-09-30,2008-09-30,2007-09-30,2006-09-30,2005-09-30
0,Cash On Hand,90943.00000,100557.00000,66301.00000,74181.00000,67155.00000,41601.00000,25077.00000,40546.00000,29129.00000,25952.00000,25620.00000,23464.00000,22111.00000,15386.00000,10110.00000,8261.00000
1,Receivables,37445.00000,45804.00000,48995.00000,35673.00000,29299.00000,30343.00000,27219.00000,20641.00000,18692.00000,11717.00000,9924.00000,5057.00000,2422.00000,1637.00000,1252.00000,895.00000
2,Inventory,4061.00000,4106.00000,3956.00000,4855.00000,2132.00000,2349.00000,2111.00000,1764.00000,791.00000,776.00000,1051.00000,455.00000,509.00000,346.00000,270.00000,165.00000
3,Pre-Paid Expenses,,,,,,,,,,,,,,,,
4,Other Current Assets,11264.00000,12352.00000,12087.00000,13936.00000,8283.00000,15085.00000,9806.00000,6882.00000,6458.00000,4529.00000,3447.00000,1444.00000,3920.00000,3805.00000,2270.00000,648.00000
5,Total Current Assets,143713.00000,162819.00000,131339.00000,128645.00000,106869.00000,89378.00000,68531.00000,73286.00000,57653.00000,44988.00000,41678.00000,31555.00000,30006.00000,21956.00000,14509.00000,10300.00000
6,"Property, Plant, And Equipment",36766.00000,37378.00000,41304.00000,33783.00000,27010.00000,22471.00000,20624.00000,16597.00000,15452.00000,7777.00000,4768.00000,2954.00000,2455.00000,1832.00000,1281.00000,817.00000
7,Long-Term Investments,100887.00000,105341.00000,170799.00000,194714.00000,170430.00000,164065.00000,130162.00000,106215.00000,92122.00000,55618.00000,25391.00000,10528.00000,2379.00000,,,
8,Goodwill And Intangible Assets,,,,,8620.00000,9009.00000,8758.00000,5756.00000,5359.00000,4432.00000,1083.00000,453.00000,492.00000,337.00000,177.00000,96.00000
9,Other Long-Term Assets,42522.00000,32978.00000,22283.00000,18177.00000,8757.00000,5422.00000,3764.00000,5146.00000,5478.00000,3556.00000,2263.00000,2011.00000,839.00000,1222.00000,1238.00000,303.00000


In [ ]:
s.query(Security.ticker).filter_by(ticker="jlkkj").first()[0]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from models import Base

In [ ]:
s.query(Base.metadata.tables['income_statement_annual'].columns.ticker).filter_by(ticker='A').all()

[('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 ('A',),
 

In [ ]:
s.close_all()

/tmp/ipykernel_68765/1419930820.py:1: SADeprecationWarning: The Session.close_all() method is deprecated and will be removed in a future release.  Please refer to session.close_all_sessions(). (deprecated since: 1.3)
  s.close_all()


In [ ]:
A = ('a','b','c')
B = ('a','b','c')
C = ('d','e','f')
D = ('g','h','i')

In [ ]:
my_list = [A,B,C]

In [ ]:
set_1 = set(my_list)
set_2 = set([C,D])

In [ ]:
len(set_1-set_2)

1